In [4]:
from abc import ABC, abstractmethod

In [5]:
# Helper fns
def mex(vals):
    """ Returns mex (minimum excluded value) of a collection """
    mex = 0
    for val in sorted(vals):
        if val is mex:
            mex += 1
    return mex

def pretty_print_grundy(grundy_pos):
    for pos, g in grundy_pos.items():
        print(f'{pos}: {g}\n')

In [6]:
# Generic class to find best moves in a game

class GenericGameCalculator(ABC):
    # A position is represented as tuple (x1,x2,x3,...) where x1 is length of bottom row, etc.
    
    def __init__(self):
        self.pos_grundy = {}
        self.pos_lose_ratio = {}

    def best_move(self, pos):
        """
        Finds the best move from the given position
        If it is possible to move to a winning position, returns the winning position
        If not, moves to the position with the greatest ratio of losing position followers
        """
        best_move = None
        best_lose_ratio = 0

        for follower in self.followers(pos):
            if self.get_grundy(follower) == 0:
                # Return follower that is winning position
                return follower
    
            lose_ratio = self.get_lose_ratio(follower)
            if lose_ratio >= best_lose_ratio:
                best_move = follower
                best_lose_ratio = lose_ratio
        return best_move

    def get_grundy(self, pos):
        """
        Gets the Sprague-Grundy value for this position
        Memoized to avoid unnecessary recomputation
        """
        pos_index = self.pos_ix(pos)
        if pos_index in self.pos_grundy:
            return self.pos_grundy[pos_index]
        else:
            result = self.compute_grundy(pos)
            self.pos_grundy[pos_index] = result
            return result
        
    def compute_grundy(self, pos):
        """
        Compute Sprague-Grundy value for this position as mex(grundy values of followers)
        """
        foll_grundy = [self.get_grundy(follower) for follower in self.followers(pos)]
        return mex(foll_grundy)
    
    def get_lose_ratio(self, pos):
        """
        Gets the lose ratio for this position: ratio of followers which are losing positions
        Memoized to avoid unnecessary recomputation
        """
        pos_index = self.pos_ix(pos)
        if pos_index in self.pos_lose_ratio:
            return self.pos_lose_ratio[pos_index]
        else:
            result = self.compute_lose_ratio(pos)
            self.pos_lose_ratio[pos_index] = result
            return result
        
    def compute_lose_ratio(self, pos):
        """
        Compute the lose ratio for this position
        """
        foll_grundy = [self.get_grundy(follower) for follower in self.followers(pos)]
        follower_win = foll_grundy.count(0)
        follower_lose = len(foll_grundy) - follower_win
        return 1 if follower_win == 0 else float(follower_lose) / (follower_win + follower_lose)
    
    def valid_move(self, pos, move):
        """ Returns true if a move is a valid follower of the original position """
        return self.pos_ix(move) in [self.pos_ix(x) for x in self.followers(pos)]
    
    @abstractmethod
    def visualize_game(self, pos):
        """ Prints the board visually to console with ascii characters """
        pass
    
    @abstractmethod
    def visualize_game_arr(self, pos):
        """ Generates an array of lines to visually represent game with ascii characters """
        pass
    
    @abstractmethod
    def followers(self, pos):
        """ Returns all direct followers of a position """
        pass
    
    @abstractmethod
    def pos_ix(self, pos):
        """ Get the string used to index this position in a dict """
        pass

    @abstractmethod
    def is_terminal(self, pos):
        """ Returns true if a position is terminal """
        pass

    @abstractmethod
    def parse_move(self, pos, inp):
        """ 
        Parses a user input move to a position
        Returns false is move is invalid syntax or invalid move in the game
        """
        pass